In [17]:
# mount google drive 

import os, sys 
from google.colab import drive

drive.mount('/content/gdrive')
%cd /content/gdrive/MyDrive/가사유사도기반추천

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/MyDrive/가사유사도기반추천


In [18]:
#!/usr/bin/env python
# coding: utf-8

# ## Dependancy
# - KoBERT
# - Python >= 3.6 
# - transformers >= 3.5
# - sentencepiece >= 0.1.6
# - Pytorch >= 1.7
# - Tensorflow >= 2.0.0

# ###0. KoBERT 설치

# KoBERT 설치 - vocab_size 8002, 600M step Training
get_ipython().system('git clone https://github.com/SKTBrain/KoBERT.git')
get_ipython().system('pip install -r ./KoBERT/requirements.txt')
get_ipython().system('pip install ./KoBERT')
get_ipython().system('ls ./KoBERT')

fatal: destination path 'KoBERT' already exists and is not an empty directory.
Processing ./KoBERT
  Created wheel for kobert: filename=kobert-0.1.2-cp37-none-any.whl size=12708 sha256=c8ad9779194f16bd0d755af34bd5b9d47a01d4665664a0a421202f94f89c3ddb
  Stored in directory: /tmp/pip-ephem-wheel-cache-t353ad32/wheels/1d/d2/2e/7356b12ceb3f85796b88b2fbd81316f05311a39058ae9aa052
Successfully built kobert
  Found existing installation: kobert 0.1.2
    Uninstalling kobert-0.1.2:
      Successfully uninstalled kobert-0.1.2


imgs  kobert  LICENSE  logs  README.md	requirements.txt  scripts  setup.py


In [19]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

from kobert.pytorch_kobert import get_pytorch_kobert_model
from kobert.utils import get_tokenizer

from tensorflow.keras.preprocessing.sequence import pad_sequences

import sentencepiece as spm

import pandas as pd
import numpy as np
import time
import datetime

In [20]:
# Check GPU
print("GPU : ",torch.cuda.get_device_name(0))

# Set GPU
device = "cuda" if torch.cuda.is_available() else "cpu"

GPU :  Tesla T4


In [21]:
# ### 1. 인풋 파이프라인 세팅

# csv 데이터셋 파일 경로
PATH = './data/remove_meaningless.csv'

# 판다스로 dataset 불러오기
dataset = pd.read_csv(PATH)

# 최대 입력 시퀀스 길이 설정
MAX_LEN = 512
# 배치 사이즈 설정
batch_size = 64

In [22]:
def make_input_from_dataset(dataset, MAX_LEN=512, batch_size=64) :
  # 구글 센텐스피스 토크나이저 소환
  tokenizer = spm.SentencePieceProcessor()

  # KoBERT 토크나이저 할당
  tokenizer.load(get_tokenizer())

  # 토크나이징
  document_bert = [tokenizer.encode_as_ids(str(i)) for i in dataset['lyrics']]

  # 맨 앞에 [CLS], 맨 뒤에 [SEP] 추가
  # [UNK]=0, [PAD]=1, [CLS]=2, [SEP]=3 in KoBERT Vocab
  for j in document_bert:
    j.insert(0,2)
    j.append(3)

  # 입력 시퀀스 만들기
  input_ids = pad_sequences(document_bert, maxlen=MAX_LEN, dtype="long", truncating="post",padding="post",value=1)

  # 어텐션 마스크 세팅
  attention_masks = []

  # Padding = 0, Not Padding = 1 / Not trained for Padding
  for seq in input_ids:
      seq_mask = [float(i!=1) for i in seq]
      attention_masks.append(seq_mask)

  # 파이토치 텐서로 변환
  dataset_inputs = torch.tensor(input_ids, dtype=torch.long)
  dataset_masks = torch.tensor(attention_masks, dtype=torch.long)

  # input_ids, attention masks 묶어서 파이토치 데이터로더로 변환
  final_data = TensorDataset(dataset_inputs, dataset_masks)
  final_dataloader = DataLoader(final_data, batch_size=batch_size)

  return final_dataloader


final_dataloader = make_input_from_dataset(dataset,MAX_LEN,batch_size)


using cached model


In [23]:
# ### 2. KoBERT


# KoBERT Model 소환
KoBERT_model, _  = get_pytorch_kobert_model()

# Pre-training된 KoBERT model에서 [CLS] vector만 쉽게 뽑을 수 있도록 클래스 재정의
class Model(nn.Module):
  global KoBERT_model # KoBERT
  def __init__(self):
    super(Model, self).__init__()
    self.bert = KoBERT_model #KoBERT
    
  def forward(self,ids, mask):
    _, pooled_output = self.bert(input_ids=ids, attention_mask=mask)
    return pooled_output

model = Model()

using cached model
using cached model


In [24]:
# 시간 표시 함수
def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))

# 최종 vectors 배열로
final_vectors = np.zeros([1,768],dtype=float)

t0 = time.time()
model.eval()

Model(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
       

In [25]:
# model gpu로 보내기
model.to(device)

# 데이터로더에서 배치만큼 반복하여 가져옴
for step, batch in enumerate(final_dataloader):
    # 경과 정보 표시
    if step % 20 == 0 and not step == 0:
        elapsed = format_time(time.time() - t0)
        print('Batch {:>5,} / {:>5,} - Time Running.. {:}.'.format(step, len(final_dataloader), elapsed))

    # 배치를 GPU에 넣음
    batch = tuple(t.to(device) for t in batch)
    
    b_input_ids, b_input_mask = batch
    
    with torch.no_grad():     
        outputs = model(b_input_ids, b_input_mask)
    
    # CPU로 데이터 이동
    vectors = outputs.detach().cpu().numpy()

    # 넘파이 배열로
    final_vectors = np.concatenate((final_vectors,vectors),axis=0)
    del vectors

print("Done.")

Batch    20 /   169 - Time Running.. 0:00:52.
Batch    40 /   169 - Time Running.. 0:01:45.
Batch    60 /   169 - Time Running.. 0:02:38.
Batch    80 /   169 - Time Running.. 0:03:32.
Batch   100 /   169 - Time Running.. 0:04:26.
Batch   120 /   169 - Time Running.. 0:05:19.
Batch   140 /   169 - Time Running.. 0:06:12.
Batch   160 /   169 - Time Running.. 0:07:06.
Done.


In [26]:
# 넘파이 배열 초기화 당시 첫행 제거
final_vectors = final_vectors[1:]

# 불러온 데이터셋이랑 최종 넘파이 배열 레코드 갯수 확인
try:
  if len(dataset) != len(final_vectors) : 
      raise Exception('Records of Dataset and Final vector shape is Not match.')
  print("최종 Context Vector Shape",final_vectors.shape)
except Exception as e:
  print('Exception is occured. ', e)



최종 Context Vector Shape (10757, 768)


In [27]:
## 최종 넘파이 배열 저장
np.save('./vec/ko_vec_rm',final_vectors)

In [28]:
print(final_vectors)

[[-0.01166219 -0.04060391 -0.46878538 ... -0.00192322 -0.02360824
  -0.0363199 ]
 [-0.04199397 -0.04072817 -0.49069458 ...  0.01971427  0.04380579
   0.02465651]
 [ 0.01055787 -0.02890142 -0.07691309 ...  0.07225733 -0.03135946
  -0.01188726]
 ...
 [-0.01127518 -0.04588813 -0.43267262 ... -0.02362569 -0.0883085
  -0.04459684]
 [ 0.01160353 -0.00202722 -0.41084278 ... -0.03555009 -0.07257079
  -0.06399281]
 [-0.00685424 -0.07076862 -0.26634115 ...  0.02750332 -0.09353186
  -0.05930189]]
